In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from numba import jit, njit, prange
from numba.types import bool_, int_, float32
from joblib import Parallel, delayed
import csv
from scipy.io.arff import loadarff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets, linear_model

In [2]:
def draw_bootstrap(X_train, y_train):
    #function for generating bootstrap samples (for bagging)
    bootstrap_indices = np.random.choice(range(len(X_train)), len(X_train), replace = True)
    X_bootstrap = np.take(X_train, bootstrap_indices, axis = 0)
    y_bootstrap = np.take(y_train,bootstrap_indices)
    return X_bootstrap, y_bootstrap

In [3]:
def regr_parameters(x,y):
    # function for estimating sigma^2 for regression  
    regr = LinearRegression(n_jobs=-1) 
    regr.fit(x,y)    
    y_pred = regr.predict(x)
    sigma_squared = mean_squared_error(y, y_pred)
    return sigma_squared

In [4]:
def find_best_split(x, y, alpha, feature, min_samples_leaf, beta=1, entropy_type = 'Renyi'):
    #function for finding the best split by one chosen attribute; 'feature' is an attribute number 
    n = y.shape[0]   #number of samples in train dataset
    thresholds = np.unique(x[:,feature])  #vector of unique values of the feature
    info_gain = np.empty(thresholds.shape[0]-1)  #form a vector for infromation gain values
    info_gain.fill(-99999)  #and fill it with large negative numbers
    
    #calculate sigma^2 before splitting 
    sigma_squared = regr_parameters(x,y)
    
    j = 0
    for split_point in thresholds[0:-1:1]: 
        #cycle by possible values of the chosen feature for splitting (without the largest value)
        #if the feature vector contains all the same values (that is, only one unique value), then splitting cannot be done 
        #building a split into left and right subtrees for a specific split_point value
        feature_vector = x[:, feature]
        split = feature_vector <= split_point
        x_left = x[split]
        y_left = y[split]
        x_right = x[np.logical_not(split)]
        y_right = y[np.logical_not(split)]
           
        #check that there are enough observations in the left and right subtrees
        if ((y_left.shape[0] >= min_samples_leaf) and (y_right.shape[0] >= min_samples_leaf)):   
            #estimate sigma^2 for left and right subtrees
            sigma_squared_left = regr_parameters(x_left,y_left)
            sigma_squared_right = regr_parameters(x_right,y_right)
            
            #calculating information gain (if alpha=1, then Shannon entropy is used; if beta=1, then Renyi entropy is used; otherwise Sharma-Mittal entropy is used)
            if (entropy_type == 'SharmaMittal') and (alpha != 1) and (beta!=1): 
                #calculating Sharma-Mittal entropy-based information gain 
                info_gain_temp = (1/(1-beta))*(1/((np.power(np.sqrt(sigma_squared),beta-1))* (np.power(np.sqrt(2*np.pi), beta-1)) * (np.power(np.sqrt(alpha),(1-beta)/(1-alpha))))-1) -((y_left.shape[0])/(y.shape[0]))* (1/(1-beta))*(1/((np.power(np.sqrt(sigma_squared_left),beta-1))* (np.power(np.sqrt(2*np.pi), beta-1)) * (np.power(np.sqrt(alpha),(1-beta)/(1-alpha))))-1) - ((y_right.shape[0])/(y.shape[0]))*(1/(1-beta))*(1/((np.power(np.sqrt(sigma_squared_right),beta-1))* (np.power(np.sqrt(2*np.pi), beta-1)) * (np.power(np.sqrt(alpha),(1-beta)/(1-alpha))))-1)
                info_gain[j] = info_gain_temp 
            else:         
                if alpha == 1:
                    #calculating Shannon entropy-based information gain
                    info_gain_temp = (1/2)*np.log(2*np.e*np.pi)+np.log(np.sqrt(sigma_squared))-((y_left.shape[0])/(y.shape[0]))*((1/2)*np.log(2*np.e*np.pi)+np.log(np.sqrt(sigma_squared_left)))-((y_right.shape[0])/(y.shape[0]))*((1/2)*np.log(2*np.e*np.pi)+np.log(np.sqrt(sigma_squared_right)))
                    info_gain[j] = info_gain_temp 
                else:
                    #calculating Renyi entropy-based information gain
                    info_gain_temp = np.log(np.sqrt(2*np.pi))+np.log(np.sqrt(sigma_squared))-(np.log(np.sqrt(alpha)))/(1-alpha)-((y_left.shape[0])/(y.shape[0]))*(np.log(np.sqrt(2*np.pi))+np.log(np.sqrt(sigma_squared_left))-(np.log(np.sqrt(alpha)))/(1-alpha))-((y_right.shape[0])/(y.shape[0]))*(np.log(np.sqrt(2*np.pi))+np.log(np.sqrt(sigma_squared_right))-(np.log(np.sqrt(alpha)))/(1-alpha))
                    info_gain[j] = info_gain_temp    
        j = j+1  
        
    
    if thresholds.shape[0] > 1:  #if feature has at least two different values, then splitting is possible
        best_ind = np.argmax(info_gain) #finding index of the best information gain
        best_thr = thresholds[best_ind]
        best_info_gain = info_gain[best_ind] #finding the best information gain
        if best_info_gain == -99999: #in this case, splitting is impossible
            return None, None
        else:
            return best_thr, best_info_gain
    else: #in this case, splitting is impossible
            return None, None

In [5]:
class DecisionTreeRenyi_and_SharmaMittal_Regressor:
    #class describing a decision tree
    def __init__(self, max_features, alpha, beta=1, max_depth=16, min_samples_split=100, min_samples_leaf=10, entropy_type = 'Renyi'):

        self._tree = {}
        self.max_features = max_features
        self._max_depth = max_depth
        self._min_samples_split = min_samples_split
        self._min_samples_leaf = min_samples_leaf
        self.alpha = alpha
        self.beta = beta
        self.entropy_type = entropy_type
        
    def _fit_node(self, sub_X, sub_y, node, depth=0):   
        if self._max_depth is not None and depth==self._max_depth: #max depth is reached
            node["type"] = "terminal"
            regr = LinearRegression(n_jobs=-1)  
            regr.fit(sub_X,sub_y)    
            node["pred"] = regr
            return    
        
        if self._min_samples_split is not None and (len(sub_y)<self._min_samples_split): #minimum number of samples is reached
            node["type"] = "terminal"
            regr = LinearRegression(n_jobs=-1) 
            regr.fit(sub_X,sub_y)    
            node["pred"] = regr
            return  
        
        feature_best, threshold_best, info_gain_best = None, None, None
        split = None
        
        #selecting a given number of features in a random manner; these features will be used for finding the best split 
        for feature in np.random.choice(sub_X.shape[1], self.max_features, replace=False):
            threshold, info_gain = find_best_split(sub_X, sub_y, self.alpha, feature, self._min_samples_leaf, self.beta, self.entropy_type)
            if info_gain is not None:
                if (info_gain_best is None) or (info_gain > info_gain_best):
                    feature_best = feature
                    info_gain_best = info_gain
                    feature_vector = sub_X[:, feature_best]
                    split = feature_vector <= threshold
                    threshold_best = threshold 
                
        if (info_gain_best is None) or (feature_best is None) or (info_gain_best==-99999):
            node["type"] = "terminal"
            regr = LinearRegression(n_jobs=-1) 
            regr.fit(sub_X,sub_y)
            node["pred"] = regr
            return   
            
        node["type"] = "nonterminal"
        node["feature_split"] = feature_best
        node["threshold"] = threshold_best
        node["left_child"], node["right_child"] = {}, {} 
            
        self._fit_node(sub_X[split], sub_y[split], node["left_child"], depth=depth+1)  
        self._fit_node(sub_X[np.logical_not(split)], sub_y[np.logical_not(split)], node["right_child"], depth=depth+1)        
        
    def _predict_node(self, x, node):  
        #here x is a single observation from test dataset
        tree = node
        pred_value = None
        while True:
            if tree['type'] == 'terminal':
                pred_value = tree['pred'].predict(x.reshape(1, -1))
                break
            else:
                feature_split_i = tree['feature_split']
                real_split_i = tree['threshold']
                if x[feature_split_i] < real_split_i:
                    tree = tree['left_child']
                else:
                    tree = tree['right_child']
        return pred_value
    
    def fit(self, X, y):
        self.shape = X.shape[1]
        self._fit_node(X, y, self._tree)
        
    def predict(self, X):
        predicted = []
        for i in range(len(X)):
            predicted.append(self._predict_node(X[i], self._tree))
        return np.array(predicted).reshape(1, -1)

In [6]:
class Renyi_and_SharmaMittal_RandomForestRegressor:
    #class describing a random forest
    def __init__(self, num_trees, max_features, alpha, beta,max_depth, min_samples_split, min_samples_leaf, entropy_type):
        
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.alpha = alpha
        self.beta = beta
        self.entropy_type = entropy_type
        self.num_trees = num_trees
        self.forest = []
        
    def fit(self, X_train, y_train):
        self.forest = Parallel(n_jobs=-1)(delayed(self.fit_tree)(X_train, y_train) for _ in range(self.num_trees))

    def fit_tree(self, X_train, y_train):
        X_bootstrap, y_bootstrap = draw_bootstrap(X_train, y_train)
        temp = DecisionTreeRenyi_and_SharmaMittal_Regressor(
            self.max_features, self.alpha, self.beta, self.max_depth, 
            self.min_samples_split, self.min_samples_leaf, self.entropy_type)
        temp.fit(X_bootstrap, y_bootstrap)
        return temp

    def predict(self, X_test):
        preds = self.forest[0].predict(X_test)
        for i in self.forest[1:]:
            preds = np.vstack([preds,i.predict(X_test)]) #Stack arrays in sequence vertically (row wise)
        preds = np.swapaxes(preds,0,1)  #row corresponds to predictions from different trees for one test sample x
        preds = np.array([np.mean(row) for row in preds]) #calcaulating mean value on all trees for each test sample х
        return preds      

In [ ]:
#example for sulfur dataset with Renyi entropy-based information gain

data = loadarff('sulfur.arff')
df = pd.DataFrame(data[0])
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='y1'), df['y1'], test_size=0.25, random_state=42)
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()
y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()

beta = 1

header = ['alpha', 'beta', 'MSE', 'MAE', 'R2']
with open('sulfur_results_Renyi_LineByLine.csv', 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for alpha in tqdm(np.arange(0.01, 1.0, 0.01)):
        model = Renyi_and_SharmaMittal_RandomForestRegressor(500, max_features = int(np.sqrt(X_train_np.shape[1])),
                alpha=alpha, beta=1, max_depth = 16, min_samples_split=120, min_samples_leaf=60,  entropy_type = 'Renyi')
        model.fit(X_train_np, y_train_np)
        preds = model.predict(X_test_np)
        MSE = mean_squared_error(y_test_np, preds)
        MAE = mean_absolute_error(y_test_np, preds)
        R2 = r2_score(y_test_np, preds)
        datax = [alpha, beta, MSE, MAE, R2]
        # write the data
        writer.writerow(datax)
        f.flush()
    


  0%|                                                                                           | 0/99 [00:00<?, ?it/s]

(7560, 6)


In [ ]:
#example for sulfur dataset with Sharma-Mittal entropy-based information gain

data = loadarff('sulfur.arff')
df = pd.DataFrame(data[0])
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='y1'), df['y1'], test_size=0.25, random_state=42)
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()
y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()

header = ['alpha', 'beta', 'MSE', 'MAE', 'R2']
with open('sulfur_results_SharmaMittal_LineByLine.csv', 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for alpha in tqdm(np.arange(0.01, 1.0, 0.01)):
        for beta in tqdm(np.arange(0.01, 1.0, 0.01)):
            model = Renyi_and_SharmaMittal_RandomForestRegressor(500, max_features = int(np.sqrt(X_train_np.shape[1])),
                alpha=alpha, beta=beta, max_depth = 16, min_samples_split=120, min_samples_leaf=60,  entropy_type = 'SharmaMittal')
            model.fit(X_train_np, y_train_np)
            preds = model.predict(X_test_np)
            MSE = mean_squared_error(y_test_np, preds)
            MAE = mean_absolute_error(y_test_np, preds)
            R2 = r2_score(y_test_np, preds)
            datax = [alpha, beta, MSE, MAE, R2]
            # write the data
            writer.writerow(datax)
            f.flush()
    